# 01_preprocess_csis_and_events.ipynb

## Preprocessing CSIS Raw Data and 2023–2024 Missile Launch Events

This notebook performs the preprocessing pipeline for the **IBM:Data Analyst Project**:
- Structuring raw CSIS missile profile data into a usable reference table
- Cleaning and standardizing the author-compiled missile launch events (2023–2024)
- Reconciling missile type classifications (e.g., SRBM, IRBM, ICBM) and technology tags (HGV)
- Producing a clean, analysis-ready dataset for subsequent EDA and visualization

### Key Outputs
- `data/processed/launch_events_2023_2024_processed.csv`
- `data/processed/data_dictionary.csv`

### Notes on Data Sources
- Original CSIS datasets are **not redistributed** in this repository due to copyright considerations.
- All processed datasets included here were independently compiled and transformed for research and educational purposes.


## 0. Setup and Project Paths

This section initializes the environment, imports required libraries,
and defines project directory paths to ensure reproducibility.


In [2]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# 노트북이 notebooks/에 있다고 가정
PROJECT_ROOT = Path("..")
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

SELF_MADE_PATH = RAW_DIR / "Self-made_2023-2024.xlsx"
CSIS_PROFILE_PATH = RAW_DIR / "CSIS_North Korea missile profile.xlsx"
CSIS_PROVOCATIONS_PATH = RAW_DIR / "CSIS_North Korean Provocations.xlsx"

print("Exists check:")
print(" - Self-made:", SELF_MADE_PATH.exists())
print(" - CSIS profile:", CSIS_PROFILE_PATH.exists())
print(" - CSIS provocations:", CSIS_PROVOCATIONS_PATH.exists())


Exists check:
 - Self-made: True
 - CSIS profile: True
 - CSIS provocations: True


## Step 1. Utility Functions

This section defines reusable helper functions used throughout the preprocessing pipeline.

The functions below handle:
- String normalization
- Automatic detection of header rows in Excel files
- Robust loading of CSIS raw Excel files with non-standard formatting

These utilities ensure that preprocessing remains reproducible and resilient to format changes in raw data sources.


In [4]:
# Step 1. Utility Functions

import re
import numpy as np
import pandas as pd

def norm_str(x):
    """Normalize string values: strip whitespace and handle NaN safely."""
    if pd.isna(x):
        return np.nan
    return str(x).strip()

def read_excel_header_autodetect(path, header_keyword="event_date"):
    """
    Read an Excel file where the header row is not necessarily the first row.
    The function detects the header row by searching for `header_keyword`
    in the first column.
    """
    raw = pd.read_excel(path, header=None)
    first_col = raw.iloc[:, 0].astype(str).str.strip()
    idx = raw.index[first_col.eq(header_keyword)].tolist()
    if not idx:
        raise ValueError(
            f"Header row not found. Cannot locate '{header_keyword}' in the first column of {path}"
        )
    header_row = idx[0]
    header = raw.iloc[header_row].tolist()
    df = raw.iloc[header_row + 1:].copy()
    df.columns = header
    df = df.dropna(how="all").reset_index(drop=True)
    return df

def find_best_header_row(df0, must_contain, search_rows=60):
    """
    Identify the most likely header row in a DataFrame by checking
    which row contains the highest number of expected column keywords.
    """
    best_row = None
    best_score = -1
    for i in range(min(search_rows, len(df0))):
        row = df0.iloc[i].astype(str).str.lower().tolist()
        score = sum(
            any(key.lower() in cell for cell in row)
            for key in must_contain
        )
        if score > best_score:
            best_score = score
            best_row = i
    return best_row

def read_csis_excel_fuzzy(path, sheet_name=0, must_contain=None):
    """
    Load CSIS raw Excel files with fuzzy header detection.
    This is necessary because CSIS Excel exports often have
    multi-row headers or notes above the actual table.
    """
    if must_contain is None:
        must_contain = ["missile", "name", "type", "class", "category", "range"]

    df0 = pd.read_excel(path, sheet_name=sheet_name, header=None)
    header_row = find_best_header_row(df0, must_contain=must_contain)

    if header_row is None:
        # Fallback: assume the first row is the header
        return pd.read_excel(path, sheet_name=sheet_name)

    header = df0.iloc[header_row].tolist()
    df = df0.iloc[header_row + 1:].copy()
    df.columns = header
    df = df.dropna(how="all").reset_index(drop=True)
    return df


## Step 2. Inspect CSIS Raw Files (Sheets & Columns)

In this step, we inspect the raw CSIS Excel files to understand their structure.
Specifically, we identify:
- Which sheets contain missile-related tables
- Which columns represent missile names and type/category information

No data transformation is performed in this step.


In [6]:
# Step 2.1 Inspect available sheets in CSIS raw files

import pandas as pd

# CSIS missile profile sheets
csis_profile_xls = pd.ExcelFile(CSIS_PROFILE_PATH)
print("[CSIS Missile Profile] Available sheets:")
for i, sheet in enumerate(csis_profile_xls.sheet_names):
    print(f"  {i}: {sheet}")

# CSIS provocations sheets (inspection only; not processed yet)
csis_prov_xls = pd.ExcelFile(CSIS_PROVOCATIONS_PATH)
print("\n[CSIS Provocations] Available sheets:")
for i, sheet in enumerate(csis_prov_xls.sheet_names):
    print(f"  {i}: {sheet}")


[CSIS Missile Profile] Available sheets:
  0: Sheet1

[CSIS Provocations] Available sheets:
  0: Sheet1


### Step 2.2 Load Candidate CSIS Profile Sheet

We now load the most likely sheet containing missile profile information.
This sheet is expected to include:
- Missile name
- Missile type or class (e.g., SRBM, IRBM, ICBM)

Because CSIS raw Excel files often contain notes or multi-row headers,
we use a fuzzy header detection loader.


In [7]:
# Step 2.2 Load a candidate CSIS profile sheet and inspect columns

# Start with the first sheet (change index if needed based on Step 2.1 output)
PROFILE_SHEET = 0

csis_profile_raw = read_csis_excel_fuzzy(
    CSIS_PROFILE_PATH,
    sheet_name=PROFILE_SHEET,
    must_contain=["missile", "name", "type", "class", "category", "range"]
)

print("Loaded CSIS profile shape:", csis_profile_raw.shape)
display(csis_profile_raw.head(10))

print("\nDetected columns:")
for col in csis_profile_raw.columns:
    print(" -", col)


Loaded CSIS profile shape: (29, 4)


,Missile Name,Class,Range,Status
0,BM-25 Musudan,IRBM,"2,500 - 4,000 km",In Development
1,Hwasong-12,IRBM,"4,500 km",In Development
2,Hwasong-13,ICBM,"5,500 - 11,500 km",Never Deployed
3,Hwasong-14,ICBM,"10,400 km",Operational
4,Hwasong-15,ICBM,"8,500 - 13,000 km",In Development
5,Hwasong-16B,IRBM,"5,500 km",Operational
6,Hwasong-17,ICBM,"15,000 km",Operational
7,Hwasong-18,ICBM,"15,000 km",Operational
8,Hwasong-19,ICBM,NaN,In development
9,Hwasong-5,SRBM,300 km,Operational



Detected columns:
 - Missile Name
 - Class
 - Range
 - Status


### Step 2.3 Sanity Check

At this stage, we expect to visually confirm:
- A column representing missile names
- A column representing missile type, class, or category

If these fields are not clearly visible, we will try a different sheet index
and repeat the inspection.


## Step 3. Build CSIS Missile Type Reference Table

In this step, we convert the CSIS missile profile table into a compact reference dataset
that maps missile names to high-level type groups (e.g., SRBM, MRBM, IRBM, ICBM, SLBM, CRUISE).

This reference will be used later to:
- validate the author-compiled event dataset
- fill missing type information when missile names match

Output:
- `data/processed/csis_missile_type_reference.csv`


In [13]:
# Step 3. Build CSIS missile type reference table

import re
import numpy as np
import pandas as pd

# 3-1) Column sanity check
expected_cols = {"Missile Name", "Class"}
missing = expected_cols - set(csis_profile_raw.columns)
if missing:
    raise ValueError(f"CSIS profile columns missing: {missing}. Found: {list(csis_profile_raw.columns)}")

# 3-2) Name normalization key (for matching with your dataset)
def norm_name_key(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("·", " ").replace("•", " ").replace("/", " ")
    s = re.sub(r"[^a-z0-9\s\-]", "", s)   # keep alnum/space/hyphen
    s = re.sub(r"\s+", " ", s).strip()
    return s

# 3-3) Normalize CSIS class into our type_group system
def normalize_type_group_from_class(x):
    if pd.isna(x):
        return np.nan
    t = str(x).upper().strip()

    # canonical groups
    if "ICBM" in t: return "ICBM"
    if "IRBM" in t: return "IRBM"
    if "MRBM" in t: return "MRBM"
    # Some sources use CRBM; in this project we fold it into SRBM for simplicity
    if "SRBM" in t or "CRBM" in t: return "SRBM"
    if "SLBM" in t: return "SLBM"
    if "CRUISE" in t: return "CRUISE"

    # If CSIS class is something else, keep NaN and handle later
    return np.nan

# 3-4) Build the reference DataFrame
ref_csis = csis_profile_raw[["Missile Name", "Class", "Range", "Status"]].copy()
ref_csis = ref_csis.rename(columns={
    "Missile Name": "missile_name_raw",
    "Class": "csis_class_raw",
    "Range": "csis_range_raw",
    "Status": "csis_status_raw",
})

# clean strings
for c in ["missile_name_raw", "csis_class_raw", "csis_range_raw", "csis_status_raw"]:
    ref_csis[c] = ref_csis[c].apply(lambda v: np.nan if pd.isna(v) else str(v).strip())

# normalized keys
ref_csis["missile_name_norm"] = ref_csis["missile_name_raw"].apply(norm_name_key)

# type group
ref_csis["type_group_base_ref"] = ref_csis["csis_class_raw"].apply(normalize_type_group_from_class)

# drop blanks and duplicates
ref_csis = ref_csis.dropna(subset=["missile_name_raw"]).drop_duplicates(subset=["missile_name_norm"]).reset_index(drop=True)

print("CSIS reference shape:", ref_csis.shape)
display(ref_csis.head(10))

print("\nType group distribution from CSIS class:")
print(ref_csis["type_group_base_ref"].value_counts(dropna=False))

# 3-5) Save (processed)
out_ref = PROCESSED_DIR / "csis_missile_type_reference.csv"
ref_csis.to_csv(out_ref, index=False, encoding="utf-8-sig")
print("\nSaved:", out_ref)


CSIS reference shape: (29, 6)


,missile_name_raw,csis_class_raw,csis_range_raw,csis_status_raw,missile_name_norm,type_group_base_ref
0,BM-25 Musudan,IRBM,"2,500 - 4,000 km",In Development,bm-25 musudan,IRBM
1,Hwasong-12,IRBM,"4,500 km",In Development,hwasong-12,IRBM
2,Hwasong-13,ICBM,"5,500 - 11,500 km",Never Deployed,hwasong-13,ICBM
3,Hwasong-14,ICBM,"10,400 km",Operational,hwasong-14,ICBM
4,Hwasong-15,ICBM,"8,500 - 13,000 km",In Development,hwasong-15,ICBM
5,Hwasong-16B,IRBM,"5,500 km",Operational,hwasong-16b,IRBM
6,Hwasong-17,ICBM,"15,000 km",Operational,hwasong-17,ICBM
7,Hwasong-18,ICBM,"15,000 km",Operational,hwasong-18,ICBM
8,Hwasong-19,ICBM,NaN,In development,hwasong-19,ICBM
9,Hwasong-5,SRBM,300 km,Operational,hwasong-5,SRBM



Type group distribution from CSIS class:
type_group_base_ref
SRBM    7
NaN     7
ICBM    6
IRBM    4
MRBM    3
SLBM    2
Name: count, dtype: int64

Saved: ..\data\processed\csis_missile_type_reference.csv


## Step 4. Process Author-Compiled Launch Event Data (2023–2024)

This step preprocesses the missile launch event dataset compiled by the author
for the period 2023–2024.

Key preprocessing rules applied:
- Separation of base missile type and technology tags (e.g., HGV)
- Explicit handling of ambiguous launch counts (e.g., '수발')
- Preservation of original descriptive fields for traceability

No external reference data is merged at this stage.


In [17]:
# Step 4. Process author-compiled launch event data (2023–2024)

# 4-1) Load author-compiled Excel (header auto-detection)
df_events_raw = read_excel_header_autodetect(
    SELF_MADE_PATH,
    header_keyword="event_date"
)

print("Loaded author data shape:", df_events_raw.shape)
display(df_events_raw.head(5))
print("\nColumns:")
print(list(df_events_raw.columns))


# 4-2) Basic cleaning and normalization
df_events = df_events_raw.copy()

# normalize string columns
for col in df_events.columns:
    if df_events[col].dtype == "object":
        df_events[col] = df_events[col].apply(norm_str)

# date and year
df_events["launch_date"] = pd.to_datetime(df_events["event_date"], errors="coerce").dt.date
df_events["year"] = pd.to_numeric(df_events.get("year"), errors="coerce").astype("Int64")

# 4-3) Handle type_group and HGV tag
# Expected pattern: e.g., "IRBM (HGV)" or "SRBM"
df_events["has_hgv"] = (
    df_events.get("type_group")
    .astype(str)
    .str.contains(r"\(HGV\)", na=False)
)

df_events["type_group_base"] = (
    df_events.get("type_group")
    .astype(str)
    .str.replace(r"\s*\(HGV\)\s*", "", regex=True)
    .str.strip()
)

# 4-4) Handle launch counts
df_events["count_launched_raw"] = df_events.get("count_launched")
df_events["count_launched"] = pd.to_numeric(
    df_events.get("count_launched"), errors="coerce"
)

# Ensure count_flag is a string dtype (nullable)
df_events["count_flag"] = pd.Series([pd.NA] * len(df_events), dtype="string")

# '수발' indicates multiple but unspecified launches → do NOT coerce to numeric
if "source_title" in df_events.columns:
    mask_multi_unspecified = df_events["source_title"].astype(str).str.contains("수발", na=False)
    df_events.loc[mask_multi_unspecified, "count_launched"] = np.nan
    df_events.loc[mask_multi_unspecified, "count_flag"] = "multiple_unspecified"


# 4-5) Missile name normalization key (for later CSIS matching)
df_events["missile_name_norm"] = df_events.get("missile_name", "").fillna("").apply(norm_name_key)

# 4-6) Time-based derived fields
dt = pd.to_datetime(df_events["launch_date"], errors="coerce")
df_events["month"] = dt.dt.month.astype("Int64")
df_events["year_month"] = dt.dt.to_period("M").astype(str)

# 4-7) Simple event_id
df_events["event_id"] = (
    dt.dt.strftime("%Y%m%d").fillna("unknown")
    + "_" + df_events["type_group_base"].fillna("UNK")
    + "_" + df_events.get("missile_name", "NA").fillna("NA")
)

# sort for readability
df_events = df_events.sort_values(["launch_date", "type_group_base"]).reset_index(drop=True)

print("\nProcessed author data preview:")
display(df_events.head(10))

# Normalize column name typo: count_lauched -> count_launched
if "count_lauched" in df_events.columns and "count_launched" not in df_events.columns:
    df_events = df_events.rename(columns={"count_lauched": "count_launched"})


Loaded author data shape: (48, 11)


,event_date,event_text,year,source_org,source_title,source_url,type_group,count_lauched,missile_name,launch_site,target
0,2023-01-01 00:00:00,"합참 “北, 평양 용성 일대서 SRBM 1발 발사”…이틀 연속 도발 감행",2023,합참,SRBM 1발 발사,siste:donga.com,SRBM,1,KN-25,평양시 룡성구역,동해
1,2023-02-18 00:00:00,"[속보] 軍 ""북, 평양순안서 장거리 추정 1발 동해상 발사"" [출처:중앙일보] h...",2023,합참,ICBM추정 1발 동해상 발사,site:joongang.co.kr,ICBM,1,화성-15,평양시 순안구역,동해
2,2023-02-20 00:00:00,"북한 ""전술핵 수단 방사포 2발 동해상으로 사격‥한미 공군에 대응""",2023,합참,SRBM 2발 발사,site:imnews.imbc.com,SRBM,2,KN-25,평안남도 숙천군,동해
3,2023-02-23 00:00:00,"北, “동해상에 순항미사일 쐈다”…우리 군, “레이더망에 포착 안 돼”",2023,국군,CRUISE 4발 발사,site:naver.com,CRUISE,4,화살-2,함경북도 김책시,동해
4,2023-03-09 00:00:00,"北, 서해방향 단거리탄도미사일 발사…한미연합연습 앞두고 도발(종합)",2023,합참,"SRBM 1발사, 이후 수발로 정정",site:yna.co.kr,SRBM,6,화성-11라,남포시 강서구역,황해



Columns:
['event_date', 'event_text', 'year', 'source_org', 'source_title', 'source_url', 'type_group', 'count_lauched', 'missile_name', 'launch_site', 'target']

Processed author data preview:


,event_date,event_text,year,source_org,source_title,source_url,type_group,count_lauched,missile_name,launch_site,target,launch_date,has_hgv,type_group_base,count_launched_raw,count_launched,count_flag,missile_name_norm,month,year_month,event_id
0,2023-01-01 00:00:00,"합참 “北, 평양 용성 일대서 SRBM 1발 발사”…이틀 연속 도발 감행",2023,합참,SRBM 1발 발사,siste:donga.com,SRBM,1,KN-25,평양시 룡성구역,동해,2023-01-01,False,SRBM,None,NaN,<NA>,kn-25,1,2023-01,20230101_SRBM_KN-25
1,2023-02-18 00:00:00,"[속보] 軍 ""북, 평양순안서 장거리 추정 1발 동해상 발사"" [출처:중앙일보] h...",2023,합참,ICBM추정 1발 동해상 발사,site:joongang.co.kr,ICBM,1,화성-15,평양시 순안구역,동해,2023-02-18,False,ICBM,None,NaN,<NA>,-15,2,2023-02,20230218_ICBM_화성-15
2,2023-02-20 00:00:00,"북한 ""전술핵 수단 방사포 2발 동해상으로 사격‥한미 공군에 대응""",2023,합참,SRBM 2발 발사,site:imnews.imbc.com,SRBM,2,KN-25,평안남도 숙천군,동해,2023-02-20,False,SRBM,None,NaN,<NA>,kn-25,2,2023-02,20230220_SRBM_KN-25
3,2023-02-23 00:00:00,"北, “동해상에 순항미사일 쐈다”…우리 군, “레이더망에 포착 안 돼”",2023,국군,CRUISE 4발 발사,site:naver.com,CRUISE,4,화살-2,함경북도 김책시,동해,2023-02-23,False,CRUISE,None,NaN,<NA>,-2,2,2023-02,20230223_CRUISE_화살-2
4,2023-03-09 00:00:00,"北, 서해방향 단거리탄도미사일 발사…한미연합연습 앞두고 도발(종합)",2023,합참,"SRBM 1발사, 이후 수발로 정정",site:yna.co.kr,SRBM,6,화성-11라,남포시 강서구역,황해,2023-03-09,False,SRBM,None,NaN,multiple_unspecified,-11,3,2023-03,20230309_SRBM_화성-11라
5,2023-03-12 00:00:00,"북한 ""어제 8·24 영웅잠수함서 전략순항미사일 2기 발사""",2023,조선중앙통신,SLCM 2기 발사,site:yna.co.kr,SLCM,2,SLCM,합경남도 홍원군,동해,2023-03-12,False,SLCM,None,NaN,<NA>,slcm,3,2023-03,20230312_SLCM_SLCM
6,2023-03-14 00:00:00,"북한, 동해로 탄도미사일 2발 발사…한미 연합훈련 반발",2023,합참,SRBM 2발 발사,site:naver.com,SRBM,2,KN-23,황해남도 장연군,동해(청진시 청암구역 방진동 피도),2023-03-14,False,SRBM,None,NaN,<NA>,kn-23,3,2023-03,20230314_SRBM_KN-23
7,2023-03-16 00:00:00,"北, 동해상으로 ICBM 발사…한일정상회담 겨냥(종합2보)",2023,합참,ICBM 1발 발사,site:yna.co.kr,ICBM,1,화성-17,평양시 순안구역,동해,2023-03-16,False,ICBM,None,NaN,<NA>,-17,3,2023-03,20230316_ICBM_화성-17
8,2023-03-19 00:00:00,"김정은, 또 핵위협…""적에 공격가할 수단…기하급수적 증대요구""(종합)",2023,조선중앙통신,전술탄도미사일 1발 발사,site:yna.co.kr,SRBM,1,KN-23,평안북도 철산군 동창리,동해,2023-03-19,False,SRBM,None,NaN,<NA>,kn-23,3,2023-03,20230319_SRBM_KN-23
9,2023-03-22 00:00:00,"[단독] 北, 해안 절벽 인근서 ‘북한판 토마호크’ 추정 미사일 발사",2023,합참,북한판 토마호크 추정 미사일 발사,site:naver.com,CRUISE,2,화살-1,함경남도 함흥시 흥남구역,동해,2023-03-22,False,CRUISE,None,NaN,<NA>,-1,3,2023-03,20230322_CRUISE_화살-1


### Step 4.1 Validation Outputs (Type Groups & Ambiguous Counts)

This section prints quick validation outputs:
1) Distribution of `type_group_base`
2) Examples of events where launch count is ambiguous (e.g., '수발' → `multiple_unspecified`)

These checks ensure preprocessing rules were applied correctly.


In [19]:
# Step 4.1 Validation outputs

print("df_events shape:", df_events.shape)

print("\n[1] type_group_base value counts:")
print(df_events["type_group_base"].value_counts(dropna=False))

print("\n[2] count_flag value counts:")
print(df_events["count_flag"].value_counts(dropna=False))

print("\n[3] Sample rows where count is ambiguous (multiple_unspecified):")
cols_show = [c for c in ["launch_date", "source_title", "count_launched_raw", "count_launched", "count_flag"] if c in df_events.columns]
display(df_events.loc[df_events["count_flag"] == "multiple_unspecified", cols_show].head(10))


df_events shape: (48, 21)

[1] type_group_base value counts:
type_group_base
SRBM         21
CRUISE       10
ICBM          6
SAT_RECON     4
SLCM          2
IRBM          2
SAM           2
UNKNOWN       1
Name: count, dtype: int64

[2] count_flag value counts:
count_flag
<NA>                    35
multiple_unspecified    13
Name: count, dtype: Int64

[3] Sample rows where count is ambiguous (multiple_unspecified):


,launch_date,source_title,count_launched_raw,count_launched,count_flag
4,2023-03-09,"SRBM 1발사, 이후 수발로 정정",None,NaN,multiple_unspecified
28,2024-01-24,CRUISE 수발 발사,None,NaN,multiple_unspecified
30,2024-01-31,CRUISE 수발 발사,None,NaN,multiple_unspecified
31,2024-02-02,CRUISE 수발 발사,None,NaN,multiple_unspecified
32,2024-02-02,SAM 수발 발사,None,NaN,multiple_unspecified
33,2024-02-14,CRUISE 수발 발사,None,NaN,multiple_unspecified
34,2024-03-18,SRBM 수발 발사,None,NaN,multiple_unspecified
36,2024-04-19,CRUISE 수발 발사,None,NaN,multiple_unspecified
37,2024-04-19,SAM 수발 발사,None,NaN,multiple_unspecified
39,2024-05-17,SRBM 수발 발사,None,NaN,multiple_unspecified


## Step 5. Reconcile Author Events with CSIS Reference

In this step, the author-compiled launch event dataset is reconciled with
the CSIS missile profile reference.

The reconciliation is performed by:
- Matching normalized missile names (`missile_name_norm`)
- Using CSIS classifications to supplement missing or inconsistent type labels
- Preserving author-defined classifications when CSIS data is not applicable

The output of this step is a finalized missile type classification
(`type_group_base_final`) for analysis.


In [20]:
# Step 5. Reconcile author events with CSIS reference

# 5-1) Merge with CSIS reference on normalized missile name
df_merged = df_events.merge(
    ref_csis[
        ["missile_name_norm", "missile_name_raw", "type_group_base_ref"]
    ].drop_duplicates("missile_name_norm"),
    on="missile_name_norm",
    how="left"
)

print("Merged shape:", df_merged.shape)

# 5-2) Create final type group
# Priority rule:
# 1) Use author-defined type_group_base if present
# 2) Else, fall back to CSIS reference type
# 3) Preserve non-missile categories (e.g., SAT_RECON, SAM, UNKNOWN)

df_merged["type_group_base_final"] = df_merged["type_group_base"]

mask_fill_from_csis = (
    df_merged["type_group_base_final"].isna()
    & df_merged["type_group_base_ref"].notna()
)

df_merged.loc[mask_fill_from_csis, "type_group_base_final"] = (
    df_merged.loc[mask_fill_from_csis, "type_group_base_ref"]
)

# Safety: do not overwrite base type with HGV
mask_ref_hgv = (
    df_merged["type_group_base_final"] == "HGV"
) & (
    df_merged["type_group_base"].notna()
)

df_merged.loc[mask_ref_hgv, "type_group_base_final"] = df_merged.loc[
    mask_ref_hgv, "type_group_base"
]

# 5-3) Final sanity check
print("\nFinal type_group_base_final value counts:")
print(df_merged["type_group_base_final"].value_counts(dropna=False))

# 5-4) Identify missile names not found in CSIS reference
missing_in_csis = (
    df_merged.loc[
        df_merged["type_group_base_ref"].isna()
        & df_merged["missile_name"].notna(),
        ["missile_name", "type_group_base"]
    ]
    .drop_duplicates()
    .sort_values("missile_name")
)

print("\nMissile names not matched in CSIS reference:")
display(missing_in_csis)


Merged shape: (48, 23)

Final type_group_base_final value counts:
type_group_base_final
SRBM         21
CRUISE       10
ICBM          6
SAT_RECON     4
SLCM          2
IRBM          2
SAM           2
UNKNOWN       1
Name: count, dtype: int64

Missile names not matched in CSIS reference:


,missile_name,type_group_base
5,SLCM,SLCM
27,UNKNOWN,IRBM
29,UNKNOWN,SLCM
40,UNKNOWN,SAT_RECON
14,UNKNOWN,SRBM
42,UNKNOWN,UNKNOWN
17,UNKNOWN,CRUISE
33,바다수리-6,CRUISE
32,번개-7,SAM
37,별찌-1-2,SAM


## Step 5A . Korean → Romanization Mapping Layer (for CSIS Matching)

This revised step incorporates manual romanization rules provided by the author:
- Hangul suffix mapping: 가/나/다/라 → A/B/C/D
- Manual name mapping for specific systems (Bulhwasal, Cheollima, etc.)
- Family rules: 화성 → Hwasong, 화살 → Hwasal, etc.

The output fields are:
- `missile_name_rom`
- `missile_name_norm_rom`
- `will_match_csis_after_rom`


In [29]:
# --- PATCH: Safe Hangul block suffix mapping (prevents 바다수리 -> 바C수리) ---

import re

HANGUL_BLOCK_MAP = {"가": "A", "나": "B", "다": "C", "라": "D"}

def map_hangul_block_suffix(s: str) -> str:
    """
    Replace Hangul block letters (가/나/다/라) with A/B/C/D ONLY when used as a suffix token.
    Rule: digit + (가/나/다/라) + (hyphen or end)
    Examples:
      - 화살-1라-3 -> 화살-1D-3
      - 화성-11라  -> 화성-11D
      - 바다수리-6 -> unchanged
    """
    if not isinstance(s, str) or not s:
        return s

    out = s
    for hangul, code in HANGUL_BLOCK_MAP.items():
        out = re.sub(rf"(\d){hangul}(?=-|$)", rf"\1{code}", out)
    return out

def romanize_missile_name(name: str) -> str:
    """
    Convert Korean missile names to CSIS-compatible romanized forms.
    Improvements:
    - Prefix-based mapping: '바다수리-6' -> 'BadaSuri-6'
    - Handles comma-separated multiple systems: '화살-1, 화살-2' -> 'Hwasal-1, Hwasal-2'
    - Keeps hangul block mapping: 가/나/다/라 -> A/B/C/D
    """
    if pd.isna(name):
        return np.nan

    s = str(name).strip()

    # Already romanized / coded (contains Latin letters) or KN-xx
    if re.search(r"[A-Za-z]", s) or re.match(r"KN-\d+", s, flags=re.IGNORECASE):
        return s

    # --- 0) Handle comma-separated multiple missile names ---
    # Examples: "화살-1, 화살-2"
    if "," in s:
        parts = [p.strip() for p in s.split(",") if p.strip()]
        rom_parts = [romanize_missile_name(p) for p in parts]  # recursive mapping
        # Remove possible NaNs
        rom_parts = [p for p in rom_parts if isinstance(p, str) and p]
        return ", ".join(rom_parts) if rom_parts else s

    # Apply hangul block mapping: 가/나/다/라 -> A/B/C/D
    s_blk = map_hangul_block_suffix(s)

    # --- 1) Manual exact mapping ---
    if s_blk in MANUAL_ROMANIZATION:
        return MANUAL_ROMANIZATION[s_blk]

    # --- 2) Manual prefix mapping (핵심 패치) ---
    # Handle 'KEY-...' patterns: e.g., '바다수리-6' / '불화살-3-31' / '천리마-1'
    for kr_key, rom_key in MANUAL_ROMANIZATION.items():
        if s_blk.startswith(kr_key + "-"):
            rest = s_blk[len(kr_key):]  # includes leading '-...'
            return rom_key + rest

    # --- 3) Family rule: 화성-XX... -> Hwasong-XX...
    m = re.match(r"^화성-?(\d+)(.*)$", s_blk)
    if m:
        num = m.group(1)
        suffix = m.group(2).strip()
        suffix = re.sub(r"\s+", "", suffix)
        suffix = suffix.replace("형", "").replace("개량", "").replace("개량형", "")
        suffix = suffix.lstrip("-")
        return f"Hwasong-{num}" + (f"-{suffix}" if suffix else "")

    # --- 4) Family rule: 화살-XX... -> Hwasal-XX...
    m = re.match(r"^화살-?(\d+)(.*)$", s_blk)
    if m:
        num = m.group(1)
        suffix = m.group(2).strip()
        suffix = re.sub(r"\s+", "", suffix).replace("형", "")
        suffix = suffix.lstrip("-")
        return f"Hwasal-{num}" + (f"-{suffix}" if suffix else "")

    # --- 5) Common standalone fallbacks ---
    if s_blk == "노동":
        return "Nodong"
    if s_blk == "무수단":
        return "Musudan"

    # Fall back: return block-mapped string
    return s_blk


In [30]:
df_events["missile_name_rom"] = df_events["missile_name"].apply(romanize_missile_name)
df_events["missile_name_norm_rom"] = df_events["missile_name_rom"].fillna("").apply(norm_name_key)

ref_keys = set(ref_csis["missile_name_norm"].dropna().unique().tolist())
df_events["will_match_csis_after_rom"] = df_events["missile_name_norm_rom"].isin(ref_keys)

unmatched = (
    df_events.loc[~df_events["will_match_csis_after_rom"], ["missile_name", "missile_name_rom", "type_group_base"]]
    .drop_duplicates()
    .sort_values(["type_group_base", "missile_name"])
)
display(unmatched)
print("Rows expected to match CSIS after romanization:",
      int(df_events["will_match_csis_after_rom"].sum()), "/", len(df_events))


,missile_name,missile_name_rom,type_group_base
17,UNKNOWN,UNKNOWN,CRUISE
33,바다수리-6,BadaSuri-6,CRUISE
28,불화살-3-31,Bulhwasal-3-31,CRUISE
9,화살-1,Hwasal-1,CRUISE
21,"화살-1, 화살-2","Hwasal-1, Hwasal-2",CRUISE
31,화살-1라-3,Hwasal-1-D-3,CRUISE
3,화살-2,Hwasal-2,CRUISE
27,UNKNOWN,UNKNOWN,IRBM
35,화성-16나,Hwasong-16-B,IRBM
32,번개-7,Pongae-7,SAM


Rows expected to match CSIS after romanization: 14 / 48


## Step 5B. Merge with CSIS Reference (Final)

This step merges the author-compiled launch events with the CSIS missile profile
using the romanized matching key.

Merge keys:
- Author data: `missile_name_norm_rom`
- CSIS reference: `missile_name_norm`

The output of this step defines the final missile type classification
used throughout analysis and visualization.


In [31]:
# Step 5B. Merge with CSIS reference using romanized key

# Merge
df_merged = df_events.merge(
    ref_csis[
        ["missile_name_norm", "missile_name_raw", "type_group_base_ref"]
    ]
    .drop_duplicates("missile_name_norm")
    .rename(columns={"missile_name_raw": "missile_name_csis"}),
    left_on="missile_name_norm_rom",
    right_on="missile_name_norm",
    how="left"
)

print("Merged shape:", df_merged.shape)

# Final type group logic
# Priority:
# 1) Keep author's type_group_base when defined (esp. SAT_RECON, SAM, SLCM)
# 2) Otherwise, fill from CSIS reference
df_merged["type_group_base_final"] = df_merged["type_group_base"]

mask_author_missing = (
    df_merged["type_group_base_final"].isna()
    | (df_merged["type_group_base_final"].astype(str).str.strip() == "")
)

mask_fill_from_csis = mask_author_missing & df_merged["type_group_base_ref"].notna()
df_merged.loc[mask_fill_from_csis, "type_group_base_final"] = (
    df_merged.loc[mask_fill_from_csis, "type_group_base_ref"]
)

# Matching flag
df_merged["matched_csis"] = df_merged["missile_name_csis"].notna()

print("\nCSIS match rate (rows):",
      int(df_merged["matched_csis"].sum()), "/", len(df_merged))

print("\nFinal type_group_base_final value counts:")
print(df_merged["type_group_base_final"].value_counts(dropna=False))

print("\nSample matched rows:")
display(
    df_merged.loc[df_merged["matched_csis"], 
                  ["launch_date", "missile_name", "missile_name_rom",
                   "missile_name_csis", "type_group_base_final"]]
    .head(10)
)

print("\nSample unmatched missile names (unique):")
unmatched_after = (
    df_merged.loc[~df_merged["matched_csis"],
                  ["missile_name", "missile_name_rom", "type_group_base"]]
    .drop_duplicates()
    .sort_values(["type_group_base", "missile_name"])
)
display(unmatched_after)


Merged shape: (48, 27)

CSIS match rate (rows): 14 / 48

Final type_group_base_final value counts:
type_group_base_final
SRBM         21
CRUISE       10
ICBM          6
SAT_RECON     4
SLCM          2
IRBM          2
SAM           2
UNKNOWN       1
Name: count, dtype: int64

Sample matched rows:


,launch_date,missile_name,missile_name_rom,missile_name_csis,type_group_base_final
0,2023-01-01,KN-25,KN-25,KN-25,SRBM
1,2023-02-18,화성-15,Hwasong-15,Hwasong-15,ICBM
2,2023-02-20,KN-25,KN-25,KN-25,SRBM
6,2023-03-14,KN-23,KN-23,KN-23,SRBM
7,2023-03-16,화성-17,Hwasong-17,Hwasong-17,ICBM
8,2023-03-19,KN-23,KN-23,KN-23,SRBM
11,2023-03-27,KN-23,KN-23,KN-23,SRBM
12,2023-04-13,화성-18,Hwasong-18,Hwasong-18,ICBM
15,2023-07-12,화성-18,Hwasong-18,Hwasong-18,ICBM
26,2023-12-18,화성-18,Hwasong-18,Hwasong-18,ICBM



Sample unmatched missile names (unique):


,missile_name,missile_name_rom,type_group_base
17,UNKNOWN,UNKNOWN,CRUISE
33,바다수리-6,BadaSuri-6,CRUISE
28,불화살-3-31,Bulhwasal-3-31,CRUISE
9,화살-1,Hwasal-1,CRUISE
21,"화살-1, 화살-2","Hwasal-1, Hwasal-2",CRUISE
31,화살-1라-3,Hwasal-1-D-3,CRUISE
3,화살-2,Hwasal-2,CRUISE
27,UNKNOWN,UNKNOWN,IRBM
35,화성-16나,Hwasong-16-B,IRBM
32,번개-7,Pongae-7,SAM


## Step 6. Export Final Processed Dataset

This step exports the finalized dataset for analysis and visualization.

Outputs (public-share safe):
- `data/processed/launch_events_2023_2024_processed.csv`
- `data/processed/data_dictionary.csv`
- `data/processed/qa_unmatched_missiles.csv` (optional QA report)

Note:
- Raw CSIS files remain local-only (not redistributed).
- Processed outputs are intended to be committed to GitHub.


In [33]:
# Step 6. Export final processed dataset

from pathlib import Path
import pandas as pd
import numpy as np

# 6-1) Ensure output directory exists
PROCESSED_DIR = Path(PROCESSED_DIR)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# 6-2) Select columns for the final dataset (only keep what we need downstream)
preferred_cols = [
    # identifiers / time
    "event_id", "launch_date", "year", "month", "year_month",
    # missile naming (traceability)
    "missile_name", "missile_name_rom", "missile_name_csis",
    # classification
    "type_group", "type_group_base", "type_group_base_ref", "type_group_base_final",
    "has_hgv",
    # counts
    "count_launched", "count_flag", "count_launched_raw",
    # location/target
    "launch_site", "target",
    # sources
    "source_org", "source_title", "source_url",
    # matching QA
    "matched_csis",
    # original text
    "event_text",
]

final_cols = [c for c in preferred_cols if c in df_merged.columns]
df_final = df_merged[final_cols].copy()

# 6-3) Standard sorting for reproducibility
# Convert launch_date for sorting (it might be python date objects)
df_final["_launch_date_dt"] = pd.to_datetime(df_final["launch_date"], errors="coerce")
df_final = df_final.sort_values(["_launch_date_dt", "type_group_base_final", "missile_name_rom"]).drop(columns=["_launch_date_dt"])
df_final = df_final.reset_index(drop=True)

print("Final dataset shape:", df_final.shape)
display(df_final.head(10))

# 6-4) Export main CSV (UTF-8 with BOM for Korean compatibility)
out_main = PROCESSED_DIR / "launch_events_2023_2024_processed.csv"
df_final.to_csv(out_main, index=False, encoding="utf-8-sig")
print("Saved:", out_main)

# 6-5) Optional QA report: unmatched missiles after romanization
qa_unmatched = (
    df_merged.loc[~df_merged["matched_csis"], ["missile_name", "missile_name_rom", "type_group_base", "type_group_base_final"]]
    .drop_duplicates()
    .sort_values(["type_group_base_final", "missile_name_rom"])
)

out_qa = PROCESSED_DIR / "qa_unmatched_missiles.csv"
qa_unmatched.to_csv(out_qa, index=False, encoding="utf-8-sig")
print("Saved QA report:", out_qa)

display(qa_unmatched)

# 6-6) Data dictionary export
desc_map = {
    "event_id": "Simple event identifier (date + type + missile)",
    "launch_date": "Launch date (YYYY-MM-DD)",
    "year": "Year",
    "month": "Month (1-12)",
    "year_month": "Year-month period (YYYY-MM)",
    "missile_name": "Missile name as recorded by the author (original)",
    "missile_name_rom": "Romanized missile name (for standardization and CSIS matching)",
    "missile_name_csis": "Closest CSIS missile name matched (if available)",
    "type_group": "Original type_group string (may include tags like (HGV))",
    "type_group_base": "Base type_group without option tags (from author)",
    "type_group_base_ref": "Type group inferred from CSIS missile profile (if matched)",
    "type_group_base_final": "Final type group used for analysis after reconciliation",
    "has_hgv": "Technology tag indicating HGV-related classification (TRUE/FALSE)",
    "count_launched": "Explicit launch count if clearly stated; NA if ambiguous (e.g., '수발')",
    "count_flag": "Flag indicating why count is NA or special handling applied",
    "count_launched_raw": "Original raw count field from the author dataset",
    "launch_site": "Launch site/location (text)",
    "target": "Target/direction/impact area (text)",
    "source_org": "Source organization label",
    "source_title": "Source title label (may include '수발')",
    "source_url": "Source URL or query hint",
    "matched_csis": "Whether this row matched a CSIS missile profile entry (TRUE/FALSE)",
    "event_text": "Original event description text",
}

data_dict = pd.DataFrame({
    "column": df_final.columns,
    "description": [desc_map.get(c, "") for c in df_final.columns]
})

out_dict = PROCESSED_DIR / "data_dictionary.csv"
data_dict.to_csv(out_dict, index=False, encoding="utf-8-sig")
print("Saved data dictionary:", out_dict)

display(data_dict)


Final dataset shape: (48, 23)


,event_id,launch_date,year,month,year_month,missile_name,missile_name_rom,missile_name_csis,type_group,type_group_base,type_group_base_ref,type_group_base_final,has_hgv,count_launched,count_flag,count_launched_raw,launch_site,target,source_org,source_title,source_url,matched_csis,event_text
0,20230101_SRBM_KN-25,2023-01-01,2023,1,2023-01,KN-25,KN-25,KN-25,SRBM,SRBM,SRBM,SRBM,False,NaN,<NA>,None,평양시 룡성구역,동해,합참,SRBM 1발 발사,siste:donga.com,True,"합참 “北, 평양 용성 일대서 SRBM 1발 발사”…이틀 연속 도발 감행"
1,20230218_ICBM_화성-15,2023-02-18,2023,2,2023-02,화성-15,Hwasong-15,Hwasong-15,ICBM,ICBM,ICBM,ICBM,False,NaN,<NA>,None,평양시 순안구역,동해,합참,ICBM추정 1발 동해상 발사,site:joongang.co.kr,True,"[속보] 軍 ""북, 평양순안서 장거리 추정 1발 동해상 발사"" [출처:중앙일보] h..."
2,20230220_SRBM_KN-25,2023-02-20,2023,2,2023-02,KN-25,KN-25,KN-25,SRBM,SRBM,SRBM,SRBM,False,NaN,<NA>,None,평안남도 숙천군,동해,합참,SRBM 2발 발사,site:imnews.imbc.com,True,"북한 ""전술핵 수단 방사포 2발 동해상으로 사격‥한미 공군에 대응"""
3,20230223_CRUISE_화살-2,2023-02-23,2023,2,2023-02,화살-2,Hwasal-2,NaN,CRUISE,CRUISE,NaN,CRUISE,False,NaN,<NA>,None,함경북도 김책시,동해,국군,CRUISE 4발 발사,site:naver.com,False,"北, “동해상에 순항미사일 쐈다”…우리 군, “레이더망에 포착 안 돼”"
4,20230309_SRBM_화성-11라,2023-03-09,2023,3,2023-03,화성-11라,Hwasong-11-D,NaN,SRBM,SRBM,NaN,SRBM,False,NaN,multiple_unspecified,None,남포시 강서구역,황해,합참,"SRBM 1발사, 이후 수발로 정정",site:yna.co.kr,False,"北, 서해방향 단거리탄도미사일 발사…한미연합연습 앞두고 도발(종합)"
5,20230312_SLCM_SLCM,2023-03-12,2023,3,2023-03,SLCM,SLCM,NaN,SLCM,SLCM,NaN,SLCM,False,NaN,<NA>,None,합경남도 홍원군,동해,조선중앙통신,SLCM 2기 발사,site:yna.co.kr,False,"북한 ""어제 8·24 영웅잠수함서 전략순항미사일 2기 발사"""
6,20230314_SRBM_KN-23,2023-03-14,2023,3,2023-03,KN-23,KN-23,KN-23,SRBM,SRBM,SRBM,SRBM,False,NaN,<NA>,None,황해남도 장연군,동해(청진시 청암구역 방진동 피도),합참,SRBM 2발 발사,site:naver.com,True,"북한, 동해로 탄도미사일 2발 발사…한미 연합훈련 반발"
7,20230316_ICBM_화성-17,2023-03-16,2023,3,2023-03,화성-17,Hwasong-17,Hwasong-17,ICBM,ICBM,ICBM,ICBM,False,NaN,<NA>,None,평양시 순안구역,동해,합참,ICBM 1발 발사,site:yna.co.kr,True,"北, 동해상으로 ICBM 발사…한일정상회담 겨냥(종합2보)"
8,20230319_SRBM_KN-23,2023-03-19,2023,3,2023-03,KN-23,KN-23,KN-23,SRBM,SRBM,SRBM,SRBM,False,NaN,<NA>,None,평안북도 철산군 동창리,동해,조선중앙통신,전술탄도미사일 1발 발사,site:yna.co.kr,True,"김정은, 또 핵위협…""적에 공격가할 수단…기하급수적 증대요구""(종합)"
9,20230322_CRUISE_화살-1,2023-03-22,2023,3,2023-03,화살-1,Hwasal-1,NaN,CRUISE,CRUISE,NaN,CRUISE,False,NaN,<NA>,None,함경남도 함흥시 흥남구역,동해,합참,북한판 토마호크 추정 미사일 발사,site:naver.com,False,"[단독] 北, 해안 절벽 인근서 ‘북한판 토마호크’ 추정 미사일 발사"


Saved: ..\data\processed\launch_events_2023_2024_processed.csv
Saved QA report: ..\data\processed\qa_unmatched_missiles.csv


,missile_name,missile_name_rom,type_group_base,type_group_base_final
33,바다수리-6,BadaSuri-6,CRUISE,CRUISE
28,불화살-3-31,Bulhwasal-3-31,CRUISE,CRUISE
9,화살-1,Hwasal-1,CRUISE,CRUISE
21,"화살-1, 화살-2","Hwasal-1, Hwasal-2",CRUISE,CRUISE
31,화살-1라-3,Hwasal-1-D-3,CRUISE,CRUISE
3,화살-2,Hwasal-2,CRUISE,CRUISE
17,UNKNOWN,UNKNOWN,CRUISE,CRUISE
35,화성-16나,Hwasong-16-B,IRBM,IRBM
27,UNKNOWN,UNKNOWN,IRBM,IRBM
37,별찌-1-2,Byeoljji-1-2,SAM,SAM


Saved data dictionary: ..\data\processed\data_dictionary.csv


,column,description
0,event_id,Simple event identifier (date + type + missile)
1,launch_date,Launch date (YYYY-MM-DD)
2,year,Year
3,month,Month (1-12)
4,year_month,Year-month period (YYYY-MM)
5,missile_name,Missile name as recorded by the author (original)
6,missile_name_rom,Romanized missile name (for standardization an...
7,missile_name_csis,Closest CSIS missile name matched (if available)
8,type_group,Original type_group string (may include tags l...
9,type_group_base,Base type_group without option tags (from author)
